In [1]:
import os
import json
import requests

In [2]:
URI = 'http://10.158.2.243:9999/'

In [3]:
t = requests.get(URI + 'api/v1/workspaces')

In [4]:
name="bolt_fastening_table_classification"

In [5]:
# ECR, S3 정보 획득
# api로 치환
s3_bucket = t.json()[0]['s3_bucket_name']
ecr_uri = t.json()[0]['ecr_base_path']
repo = name
ws_name = t.json()[0]['name']


In [6]:
#alo 제작을 위한 코드복사 (임시)
!rm -rf ./alo
!mkdir ./alo
!cp ../../main.py ../../assets/ ../../src/ ../../config/ ./alo -rf
!cp ./origin/Dockerfile ./origin/Makefile ./ -rf

In [7]:
# train infernece를 세팅해줘야 한다
pipeline = "train"
# pipeline = "infenrence"
# !sed -i "s/^NS=.*/NS={ecr_uri}/" ./Makefile
!sed -i "s/^NS=.*$/NS={ecr_uri.split("/")[1]}/" Makefile
!sed -i 's/^REPO=.*$/REPO={repo}/' Makefile
!sed -i 's/^TYPE=.*$/TYPE={pipeline}/' ./Makefile
!sed -i 's/^SOLUTION_PIPELINE_MODE=.*$/SOLUTION_PIPELINE_MODE='{pipeline}'/' ./Dockerfile

In [8]:
#기존에 이미지가 올라와 있다면 제거 하고 ecr을 업로드 합니다
!make ecr

aws ecr get-login-password --region ap-northeast-2 | docker login --username AWS --password-stdin 086558720570.dkr.ecr.ap-northeast-2.amazonaws.com
WARNING! Your password will be stored unencrypted in /home/ruci.sung/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
make: Circular ecr <- ecr dependency dropped.
aws ecr create-repository --repository-name acp-kubeflow-lhs/train/bolt_fastening_table_classification --image-scanning-configuration scanOnPush=true --region ap-northeast-2

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'acp-kubeflow-lhs/train/bolt_fastening_table_classification' already exists in the registry with id '086558720570'
Makefile:24: recipe for target 'ecr' failed
make: *** [ecr] Error 255


In [ ]:
!make sync

In [ ]:
get_response = requests.get('http://10.158.2.243:9999/api/v1/rass')
get_response = get_response.json()

In [ ]:
params = {"name":get_response['rass'][2]['name'],
"ras_version_id":get_response['rass'][2]['version']['id'],
"workspace_name":"cism-ws"
}

In [ ]:
cass = requests.post('http://10.158.2.243:9999/api/v1/cass', params=params)
cass.json()

In [ ]:
data = {
    "name":cass.json()['name'],
    "cas_id":cass.json()['id']
}

In [ ]:
requests.post('http://10.158.2.243:9999/api/v1/stream', params=data)

In [ ]:
data = {
    "workspace_name": "cism-ws"
}

In [ ]:
stream = requests.get('http://10.158.2.243:9999/api/v1/stream', data)
stream.json()

In [ ]:
stream.json()['streams'][10]

In [ ]:
# 솔메를 읽어서 metadata_json에 넣기
# data = {
#   "metadata_json": sm.sm_yaml,
#   "config_path": ""
# }
# data =json.dumps(data)

In [ ]:
pipeline = requests.post('http://10.158.2.243:9999/api/v1/stream/{e4b9771d-d130-492e-ba24-c9d1b6b3b449}/start', data=data)
pipeline.json()

In [ ]:
	
info = requests.get('http://10.158.2.243:9999/api/v1/stream/{1564b13b-fe78-4ee1-9a26-404ff2244fc7}/info')
info.json()